In [1]:
import time
import asyncio

import math
import numpy as np

from PIL import Image, ImageDraw, ImageFont

from rgbmatrix import RGBMatrix, RGBMatrixOptions

import apds9930
apds9930.APDS9930_IDs = [0x29, 0x39]
sensor = apds9930.APDS9930(1)

In [2]:
BRIGHTNESS = 50
class Tablo:
    
    def __init__(self, brightness = BRIGHTNESS, widgets = []):
        options = RGBMatrixOptions()
        options.drop_privileges = False
        options.rows = 64
        options.cols = 128
        options.chain_length = 2
        options.led_rgb_sequence = 'RBG'
        options.panel_type = 'FM6126A'
        options.hardware_mapping = 'regular'
        options.gpio_slowdown = 2
        options.pwm_bits = 11
        options.pwm_dither_bits = 2
        options.pwm_lsb_nanoseconds = 50
        
        self.matrix = RGBMatrix(options = options)
        self.matrix.brightness = brightness
        self.image = Image.new('RGB', (256,64), 'black')
        self.widgets = [w.start() for w in widgets]
        
    def update(self, brightness = None):
        if brightness is not None:
            self.matrix.brightness = brightness
        self.matrix.SetImage(self.image)
    def clear(self):
        self.image = Image.new('RGB', (256,64), 'black')
        self.update()
        


class Widget():
    created_list=[] #makes Widgets unable to delete, but makes Widget.stop_all() work
    def stop_all():
        for w in Widget.created_list:
            if w.task:
                w.stop()
    def __init__(self, size, position = (0,0)):
        Widget.created_list.append(self)
        self.w, self.h = self.size = size
        self.x, self.y = self.position = position
        self._running=False
        self.task=None
        
        self.image = Image.new("RGB", self.size, '#000000')
        self.draw = ImageDraw.Draw(self.image)
        
    def clear(self):
        self.draw.rectangle((0,0)+self.size, fill = '#000000')
        #self.place()
    def place(self):
        tablo.image.paste(self.image, box=self.position)
        tablo.update()
    def update_frame(self):
        pass
        #to be defined in subclasses individually
    async def _start(self, rr = 5):
        self._running = True
        self.update_frame()
        while self._running:
            self.place()
            await asyncio.sleep(rr - time.time() % rr)
            self.update_frame()
    def start(self, *args, **kwargs):
        assert(not self._running)
        self.task = asyncio.create_task(self._start(*args, **kwargs))
        self.task.name = type(self)
        return self
    def stop(self):
        self._running = False
        self.task.cancel()        
        
        
try:
    del(tablo)
    Widget.stop_all()
except:
    pass
tablo = Tablo()

In [3]:
try:
    del(tablo)
    Widget.stop_all()
except:
    pass

# time.sleep(5)

tablo = Tablo(brightness=20)

tablo.matrix.SetImage(
      Image.new('RGB', (256,64),'#ffffff'), 0,0
      )

tablo.matrix.SetImage(
      Image.new('RGB', (64,64),'#000000'), 128,32
      )

# tablo.matrix.SetImage(
#       Image.new('RGB', (64,64),'#ffffff'), 64*3,0
# )

In [4]:
class Test_W(Widget):
    colors = ["yellow" ,"red", "green", "blue", "white"]
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.n = 0
    def update_frame(self):
        y = self.n % self.h
        color = self.colors[self.n//self.h]
        self.draw.line((0, y,self.w, y), fill=color)
        self.n += 1
        self.n %= 64*len(self.colors)

async def run_test():
    example = Test_W((256, 64))
    example.start(rr=0.001)
    await asyncio.sleep(3)
    example.stop()
    tablo.clear()
asyncio.create_task(run_test())

<Task pending coro=<run_test() running at <ipython-input-4-d42636c0eff8>:13>>

In [5]:
DEFAULT_FONT = '/home/pi/Hack-Regular.ttf'
class Time_W(Widget):
    def __init__(self, *args,
                 timezone=+3, t_format="%p %I:%M:%S",
                 font_name=DEFAULT_FONT, font_color="#ffffff",
                 **kwargs):
        super().__init__(*args, **kwargs)
        self.timezone = timezone
        self.t_format = t_format
        self.font_obj = ImageFont.truetype(font_name, self.h)
        self.font_color = font_color
    def update_frame(self):
        self.clear()
        t = time.time() + 3600*self.timezone
        t = time.gmtime(t)
        t_str = time.strftime(self.t_format, t)
        self.draw.multiline_text((0,0), t_str, fill=self.font_color, font=self.font_obj)

In [6]:
class Time_Arrows_W(Widget):
    def __init__(self, *args, timezone=+3, **kwargs):
        super().__init__(*args, **kwargs)
        self.timezone = timezone
    def update_frame(self):
        self.clear()
        t = time.time()
        r = min(self.w, self.h)/2
        for i in range(12):
            x1 = r + r*math.cos(math.pi*i/6)
            y1 = r + r*math.sin(math.pi*i/6)
            x2 = r + 0.93*r*math.cos(math.pi*i/6)
            y2 = r + 0.93*r*math.sin(math.pi*i/6)
            fill="#ffffff" if i%3==0 else "#990000"
            self.draw.line((x1,y1,x2,y2), fill=fill)
        
        hx =   0.6*r*math.sin(math.pi*( (t/3600)%24+self.timezone )/6)
        hy = - 0.6*r*math.cos(math.pi*( (t/3600)%24+self.timezone )/6)
        self.draw.line((r,r,r+hx,r+hy), fill="#ffffff")
        
        mx =   0.9*r*math.sin(math.pi*( (t/60)%60 )/30)
        my = - 0.9*r*math.cos(math.pi*( (t/60)%60 )/30)
        self.draw.line((r,r,r+mx,r+my), fill="#ffffff")
        
        sx =   r*math.sin(math.pi*(t%60)/30)
        sy = - r*math.cos(math.pi*(t%60)/30)
        self.draw.line((r+sx,r+sy,r+0.95*sx,r+0.95*sy), fill="#ffffff")

In [7]:
class Weather_W(Widget):
    def __init__(self, *args,
                 location=+3, w_format="",
                 font_name=DEFAULT_FONT, font_color="#ffffff",
                 **kwargs):
        super().__init__(*args, **kwargs)
        self.location = location
        self.w_format = w_format
        self.font_obj = ImageFont.truetype(font_name, self.h)
        self.font_color = font_color
    def update_frame(self):
        self.clear()
        
        #w = 
        t_str = time.strftime(self.t_format, t)
        self.draw.text((0,0), t_str, fill=self.font_color, font=self.font_obj)

In [8]:
Widget.stop_all()
tablo.clear()
hight=18
#time_bashk = Time_W(256, hight, timezone=+5, t_format="%I:%M %p  %d/%m/%y")
time_msk   = Time_W((256, hight), timezone=+3, t_format="%p %I:%M:%S", font_color='#ffffff')
date_msk   = Time_W((256, hight), (0,hight), timezone=+3, t_format="%a %d/%m", font_color='#ffffff')

#time_cdt   = Time_W((1602*, hight), (0, hight*2), timezone=-5, t_format="%I:%M %p  CDT", font_color='#eeeeee')

#time_bashk.start(rr = 1)
time_msk.start(rr = 1)
date_msk.start(rr = 5)
#time_cdt.start(rr = 1)

analog=Time_Arrows_W((50,50), (144,0), timezone = +5)
#analog.start(refresh_rate = 1)

In [10]:
class BrightnessScheduler():
    def __init__(self):
        self.task = None
        self.history = np.zeros((20,))
    async def _start(self):       
        pass       #тут мозги
    def start(self):
        self.task = asyncio.create_task

In [11]:
sensor = apds9930.APDS9930(1)
sensor.set_mode(apds9930.ALL, True)
for i in range(30):
    time.sleep(.33)
    print('%.2f, %.2f' % (sensor.ambient_light, sensor.proximity), sep='', end='\r')

In [12]:
tablo.update(100)
async def f():
    while time.localtime() < time.strptime('Sat Mar 13 9:00:00 2021'):
        await asyncio.sleep(1)
    tablo.update(45)
    
    while time.localtime() < time.strptime('Sat Mar 13 19:00:00 2021'):
        await asyncio.sleep(1)
    tablo.update(30)
    
    while time.localtime() < time.strptime('Sat Mar 13 22:00:00 2021'):
        await asyncio.sleep(1)
    tablo.update(10)
    
    
    while time.localtime() < time.strptime('Sun Mar 14 00:20:00 2021'):
        await asyncio.sleep(1)
    tablo.update(3)
    
task2 = asyncio.create_task(f())

In [13]:
def change_brightness(b: int) -> None:
    assert 1<=b<=100, "Ну от 1 до 100 просил же!"
    assert type(b) is int, "Ну просил же целое число!"

    tablo.update(b)

In [73]:
change_brightness(3)